# Generate WRDS CRSP SQL Queries

In [12]:
import numpy as np
import pandas as pd

from datetime import datetime

In [3]:
DATA_PATH = "../data/"

In [17]:
senate = pd.read_csv(DATA_PATH + "senate_all_transactions.csv")
senate[:5]

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date
0,08/10/2022,Spouse,--,"Parker-Hannifin Corp 701094AQ7 <div class=""tex...",Corporate Bond,Purchase,"$15,001 - $50,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/7...,09/16/2022
1,08/01/2022,Self,IVV,iShares Core S&amp;P 500 ETF,Stock,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/7...,09/15/2022
2,08/01/2022,Self,EFAV,iShares MSCI EAFE Min Vol Factor ETF,Stock,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/7...,09/15/2022
3,08/01/2022,Self,XLY,SPDR Select Sector Fund - Consumer Discretionary,Stock,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/7...,09/15/2022
4,08/01/2022,Self,VTRS,Viatris Inc. - Common Stock,Stock,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/7...,09/15/2022


In [28]:
senate_dt_tic = senate[["transaction_date", "ticker"]].copy()

date_q = list(map(lambda x: datetime.strptime(x, '%m/%d/%Y').strftime("%Y-%m-%d"), senate_dt_tic["transaction_date"]))
ticker_q = senate_dt_tic["ticker"].replace("--", np.nan)

senate_dt_tic["date_q"] = date_q
senate_dt_tic["ticker_q"] = ticker_q

senate_dt_tic

,transaction_date,ticker,date_q,ticker_q
0,08/10/2022,--,2022-08-10,NaN
1,08/01/2022,IVV,2022-08-01,IVV
2,08/01/2022,EFAV,2022-08-01,EFAV
3,08/01/2022,XLY,2022-08-01,XLY
4,08/01/2022,VTRS,2022-08-01,VTRS
...,...,...,...,...
9548,08/17/2012,NaN,2012-08-17,NaN
9549,08/16/2012,NaN,2012-08-16,NaN
9550,08/15/2012,NaN,2012-08-15,NaN
9551,08/02/2012,NaN,2012-08-02,NaN


In [51]:
senate_dt_tic_grouped = pd.DataFrame(senate_dt_tic.groupby("ticker_q")["date_q"].apply(list))
senate_dt_tic_grouped["date_q_str"] = senate_dt_tic_grouped["date_q"].astype(str).str.replace("[", "(", regex=False)
senate_dt_tic_grouped["date_q_str"] = senate_dt_tic_grouped["date_q_str"].str.replace("]", ")", regex=False)
senate_dt_tic_grouped = senate_dt_tic_grouped.reset_index()
senate_dt_tic_grouped

,ticker_q,date_q,date_q_str
0,0QZI.IL,"[2019-09-25, 2019-09-25]","('2019-09-25', '2019-09-25')"
1,3V64.TI,"[2019-09-25, 2019-09-25]","('2019-09-25', '2019-09-25')"
2,A,"[2016-09-13, 2015-08-24, 2015-02-24, 2014-10-02]","('2016-09-13', '2015-08-24', '2015-02-24', '20..."
3,AA,"[2021-10-18, 2021-10-15, 2021-07-29, 2021-07-1...","('2021-10-18', '2021-10-15', '2021-07-29', '20..."
4,AAGIY,[2019-03-11],('2019-03-11')
...,...,...,...
1169,ZNGA,"[2016-12-15, 2016-04-20, 2015-01-16, 2014-11-07]","('2016-12-15', '2016-04-20', '2015-01-16', '20..."
1170,ZNGA.SW,[2017-07-14],('2017-07-14')
1171,ZTS,"[2015-01-14, 2015-01-29, 2015-01-30]","('2015-01-14', '2015-01-29', '2015-01-30')"
1172,^MWE,[2015-12-04],('2015-12-04')


In [50]:
def generate_sql_queries(ticker, date_tuple):
    return f"SELECT * FROM crsp.wrds_dsfv2_query WHERE ticker='{ticker}' AND dlycaldt IN {date_tuple};"
v_generate_sql_queries = np.vectorize(generate_sql_queries)

In [52]:
senate_dt_tic_grouped["sql"] = v_generate_sql_queries(senate_dt_tic_grouped["ticker_q"], senate_dt_tic_grouped["date_q_str"])
senate_dt_tic_grouped

,ticker_q,date_q,date_q_str,sql
0,0QZI.IL,"[2019-09-25, 2019-09-25]","('2019-09-25', '2019-09-25')",SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
1,3V64.TI,"[2019-09-25, 2019-09-25]","('2019-09-25', '2019-09-25')",SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
2,A,"[2016-09-13, 2015-08-24, 2015-02-24, 2014-10-02]","('2016-09-13', '2015-08-24', '2015-02-24', '20...",SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
3,AA,"[2021-10-18, 2021-10-15, 2021-07-29, 2021-07-1...","('2021-10-18', '2021-10-15', '2021-07-29', '20...",SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
4,AAGIY,[2019-03-11],('2019-03-11'),SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
...,...,...,...,...
1169,ZNGA,"[2016-12-15, 2016-04-20, 2015-01-16, 2014-11-07]","('2016-12-15', '2016-04-20', '2015-01-16', '20...",SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
1170,ZNGA.SW,[2017-07-14],('2017-07-14'),SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
1171,ZTS,"[2015-01-14, 2015-01-29, 2015-01-30]","('2015-01-14', '2015-01-29', '2015-01-30')",SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...
1172,^MWE,[2015-12-04],('2015-12-04'),SELECT * FROM crsp.wrds_dsfv2_query WHERE tick...


In [55]:
# test
senate_dt_tic_grouped.iloc[2, 3]

"SELECT * FROM crsp.wrds_dsfv2_query WHERE ticker='A' AND dlycaldt IN ('2016-09-13', '2015-08-24', '2015-02-24', '2014-10-02');"

In [56]:
output = senate_dt_tic_grouped[["ticker_q", "sql"]].copy()
output.to_csv("sql_queries.csv", index=False)